# Spectral Health v0.2: Многоканальная Диагностика

Этот ноутбук демонстрирует новые возможности версии 0.2:
1. Обучение профиля на "здоровых" данных (несколько каналов).
2. Сохранение профиля.
3. Детекция аномалий на новых данных.
4. Генерация отчета.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from spectral_physics import (
    timeseries_to_spectrum,
    build_health_profile,
    save_health_profile,
    load_health_profile,
    generate_markdown_report,
    spectral_band_power,
    spectral_entropy
)

# Параметры симуляции
dt = 0.001
t = np.arange(0, 1.0, dt)

## 1. Генерация обучающих данных (Здоровое состояние)

Симулируем два канала:
- **Motor X**: 50 Гц (основная) + шум
- **Motor Y**: 50 Гц + 100 Гц (гармоника) + шум

In [ ]:
def generate_healthy_x():
    return np.sin(2*np.pi*50*t) + 0.1*np.random.randn(len(t))

def generate_healthy_y():
    return np.sin(2*np.pi*50*t) + 0.5*np.sin(2*np.pi*100*t) + 0.1*np.random.randn(len(t))

# Генерируем по 3 примера на канал
train_x = [timeseries_to_spectrum(generate_healthy_x(), dt=dt) for _ in range(3)]
train_y = [timeseries_to_spectrum(generate_healthy_y(), dt=dt) for _ in range(3)]

training_data = {
    "motor_x": train_x,
    "motor_y": train_y
}

## 2. Обучение профиля

In [ ]:
print("Building profile...")
profile = build_health_profile(training_data)

# Сохраняем и загружаем для проверки
save_health_profile(profile, "demo_profile.npz")
loaded_profile = load_health_profile("demo_profile.npz")
print("Profile saved and loaded successfully.")

## 3. Симуляция аномалии

- **Motor X**: OK
- **Motor Y**: Появилась частота 250 Гц (дефект подшипника)

In [ ]:
current_x = timeseries_to_spectrum(generate_healthy_x(), dt=dt)

# Аномалия в Y
signal_y_bad = generate_healthy_y() + 0.4 * np.sin(2*np.pi*250*t)
current_y = timeseries_to_spectrum(signal_y_bad, dt=dt)

current_data = {
    "motor_x": current_x,
    "motor_y": current_y
}

## 4. Диагностика

In [ ]:
thresholds = {
    "motor_x": 0.1,
    "motor_y": 0.15
}

scores = profile.score(current_data)
anomalies = profile.is_anomalous(current_data, thresholds)

print("Results:")
for ch in scores:
    status = "ANOMALY" if anomalies[ch] else "OK"
    print(f"{ch}: dist={scores[ch]:.4f}, status={status}")

## 5. Расширенные метрики

Посмотрим на энтропию и мощность в полосе дефекта (240-260 Гц).

In [ ]:
band_power = spectral_band_power(current_y, freq_min=240, freq_max=260)
entropy = spectral_entropy(current_y)

print(f"Motor Y Band Power (240-260 Hz): {band_power:.4f}")
print(f"Motor Y Entropy: {entropy:.4f}")

## 6. Генерация отчета

In [ ]:
generate_markdown_report(scores, thresholds, "demo_report.md")
print("Report generated: demo_report.md")